In [1]:
using StatisticalRethinking, CmdStan, StanMCMCChain, JLD
gr(size=(500,500));

ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

df2 = filter(row -> row[:age] >= 18, df)
mean_height = mean(df2[:height])
df2[:height_c] = convert(Vector{Float64}, df2[:height]) .- mean_height
first(df2, 5)

max_height_c = maximum(df2[:height_c])
min_height_c = minimum(df2[:height_c])

heightsmodel = "
// Inferring a Rate
data {
  int N;
  real h[N];
}
parameters {
  real<lower=0> sigma;
  real<lower=$(min_height_c),upper=$(max_height_c)> mu;
}
model {
  // Priors for mu and sigma
  mu ~ normal(178, 20);
  sigma ~ uniform( 0 , 50 );

  // Observed heights
  h ~ normal(mu, sigma);
}
";

stanmodel = Stanmodel(name="heights", monitors = ["mu", "sigma"],model=heightsmodel,
  output_format=:mcmcchain);

heightsdata = Dict("N" => length(df2[:height]), "h" => df2[:height_c]);

rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);

describe(chn)

JLD.save("m4.2s.jld",
  "range", chn.range,
  "a3d", chn.value,
  "names", chn.names,
  "chains", chn.chains)

d = JLD.load(joinpath(ProjDir, "m4.2s.jld"))

chn2 = MCMCChain.Chains(d["a3d"], names=d["names"])

describe(chn2)


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/heights.stan will be updated.


Inference for Stan model: heights_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.031, 0.035, 0.033, 0.030) seconds, 0.13 seconds total
Sampling took (0.058, 0.050, 0.059, 0.046) seconds, 0.21 seconds total

                 Mean     MCSE  StdDev     5%    50%   95%    N_Eff  N_Eff/s    R_hat
lp__             -932  2.3e-02    0.99   -934   -932  -931  1.9e+03  8.9e+03  1.0e+00
accept_stat__    0.92  1.5e-03    0.10   0.71   0.96   1.0  4.9e+03  2.3e+04  1.0e+00
stepsize__       0.87  2.7e-02   0.038   0.83   0.87  0.93  2.0e+00  9.4e+00  2.2e+13
treedepth__       2.0  3.9e-02    0.68    1.0    2.0   3.0  3.0e+02  1.4e+03  1.0e+00
n_leapfrog__      4.8  3.0e-01     5.6    1.0    3.0    15  3.5e+02  1.6e+03  1.0e+00
divergent__      0.00      nan    0.00   0.00   0.00  0.00      nan      nan      nan
energy__      

end of m4.2s#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*